In [1]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

import torch 
import tiktoken

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
len(text)

1115394

In [4]:
## Easy way to encode text

def normal_encode(text):
    chars = sorted(list(set(text)))
    vocab_size = len(chars)
    
    # create a mapping from characters to integers
    stoi = {ch:i for i, ch in enumerate(chars)}
    itos = {i:ch for i, ch in enumerate(chars)}
    encode = lambda s: [stoi[c] for c in s] # encode string to list of integers, each integer was a charater encoded
    decode = lambda l: ''.join([itos[i] for i in l]) # decode list of integers to string 

    return vocab_size, encode, decode

In [5]:
vocab_size, encode, decode = normal_encode(text)
data = torch.tensor(encode(text), dtype=torch.long)

# enc = tiktoken.get_encoding('gpt2')
# vocab_size, encode = enc.n_vocab, enc.encode(text)
# data = torch.tensor(encode, dtype=torch.long)

# print(data.shape, data.type)
# print(data[:1000])

In [6]:
# split up data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8
train_data[:block_size+1]
"""
res: tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252])
In chunk of nine charaters have eight individual predictions of position. 
Like: 5962 with 22307; 5962, 22307 with 25, ... 
That's why we plus 1 in each block size
"""

"\nres: tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252])\nIn chunk of nine charaters have eight individual predictions of position. \nLike: 5962 with 22307; 5962, 22307 with 25, ... \nThat's why we plus 1 in each block size\n"

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1] # target of each positions
for index_target in range(block_size):
    context = x[:index_target+1]
    target = y[index_target]
    print("When input is {0} the target : {1}".format(context, target))

When input is tensor([18]) the target : 47
When input is tensor([18, 47]) the target : 56
When input is tensor([18, 47, 56]) the target : 57
When input is tensor([18, 47, 56, 57]) the target : 58
When input is tensor([18, 47, 56, 57, 58]) the target : 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target : 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target : 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target : 58


In [9]:
torch.manual_seed(1337)
block_size = 8 # maximum context length for predictions
batch_size = 4 # number of independent sequences which process parallel

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("Inputs: {0}, shape: {1}".format(xb, xb.shape))
print("Targets: {0}, shape: {1}".format(yb, yb.shape))
print("-----")

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
#         print("When input is {0} the target: {1}".format(context.tolist(), target))
        

Inputs: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]), shape: torch.Size([4, 8])
Targets: tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]]), shape: torch.Size([4, 8])
-----


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [11]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # Batch, Time, Channel (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C).to(device)
            targets = targets.view(B*T).to(device)
            loss = F.cross_entropy(logits, targets).to(device)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        #idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions 
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get posibilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
# print(enc.decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, device='cuda:0', grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [12]:
## Train model

# create a Pytorch optimizer
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)


In [13]:
batch_size = 32
for step in range(100):
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    print(loss.item())

4.704006671905518
4.721129894256592
4.6532135009765625
4.70629358291626
4.780948638916016
4.751323699951172
4.839620113372803
4.668044090270996
4.7438063621521
4.774146556854248
4.690934658050537
4.789270877838135
4.617897033691406
4.651078701019287
4.886624813079834
4.703958034515381
4.757771968841553
4.655284881591797
4.709480285644531
4.674715042114258
4.7607269287109375
4.789508819580078
4.6539740562438965
4.662156105041504
4.673258304595947
4.666038513183594
4.730401039123535
4.755608558654785
4.7124924659729
4.745827674865723
4.7270827293396
4.735453128814697
4.777830600738525
4.64369010925293
4.665538311004639
4.798058032989502
4.7178120613098145
4.68403959274292
4.819314956665039
4.614159107208252
4.574166774749756
4.561123371124268
4.816129207611084
4.60658073425293
4.620136260986328
4.725918769836426
4.650966167449951
4.594605922698975
4.720812797546387
4.699873924255371
4.672942638397217
4.728536605834961
4.662074089050293
4.617297172546387
4.600391387939453
4.65391492843627

In [14]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))



xiKi-RJ:CgqVuUa!U?qMH.uk!sCuMXvv!CJFfx;LgRyJknOEti.?I&-gPlLyulId?XlaInQ'q,lT$
3Q&sGlvHQ?mqSq-eON
x?SP fUAfCAuCX:bOlgiRQWN:Mphaw
tRLKuYXEaAXxrcq-gCUzeh3w!AcyaylgYWjmJM?Uzw:inaY,:C&OECW:vmGGJAn3onAuMgia!ms$Vb q-gCOcPcUhOnxJGUGSPJWT:.?ujmJFoiNL&A'DxY,prZ?qdT;hoo'dHooXXlxf'WkHK&u3Q?rqUi.kz;?Yx?C&u3Qbfzx


In [15]:
# mathematical trick in self-attention

torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])